!pip install pingouin

# FUNCTIONS

## variables and paths

In [ ]:
file_path = r"C:\Users\LEGION\OneDrive\Maghale\Vaccination in dialysi\C19VacEfic - Submit - Yasaman\E12_clean_labeled_colored_4sharing.csv"

figure_destination_path = r"C:\Users\LEGION\OneDrive\Maghale\Vaccination in dialysi\E3 Submit\Figures From Python"

## figure & stat

### Save figure

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

def save_figure(plot, destination_path, figure_name, save_PDF=True, save_JPEG=True, save_PNG=True, dpi=300):
    """
    Save a plot in PDF, PNG, and JPEG formats with specified options.

    Parameters:
    plot: matplotlib plot object
        The plot to be saved.
    destination_path: str
        The directory path where the plot will be saved.
    figure_name: str
        The base name of the saved figure file.
    save_PDF: bool, optional
        If True, save the plot as a PDF file. Default is True.
    save_JPEG: bool, optional
        If True, save the plot as a JPEG file. Default is True.
    save_PNG: bool, optional
        If True, save the plot as a PNG file. Default is True.
    dpi: int, optional
        The resolution in dots per inch for the saved plot. Default is 300.
    """

    
    # Ensure the destination path ends with a slash
    if not destination_path.endswith('/'):
        destination_path += '/'

    # Save as PDF
    if save_PDF:
        pdf_path = f"{destination_path}{figure_name}.pdf"
        plot.savefig(pdf_path, format='pdf', dpi=dpi, bbox_inches='tight')
        print(f"Figure saved as PDF: {pdf_path}")

    # Save as JPEG
    if save_JPEG:
        jpeg_path = f"{destination_path}{figure_name}.jpeg"
        plot.savefig(jpeg_path, format='jpeg', dpi=dpi, bbox_inches='tight')
        print(f"Figure saved as JPEG: {jpeg_path}")

    # Save as PNG
    if save_PNG:
        png_path = f"{destination_path}{figure_name}.png"
        plot.savefig(png_path, format='png', dpi=dpi, bbox_inches='tight')
        print(f"Figure saved as PNG: {png_path}")

# Example usage
# Create a sample plot
#fig, ax = plt.subplots()
#ax.plot([0, 1, 2], [0, 1, 4])

# Save the plot
#save_figure(fig, "path/to/save", "sample_plot")

### Stat: bootstrap, pairwise t-test wilcoxon, chi-suqre, p-value transform

In [ ]:
def calc_95ci_ind(data1, data2):
    mean1 = np.mean(data1)
    mean2 = np.mean(data2)
    se1 = stats.sem(data1)
    se2 = stats.sem(data2)
    mean_diff = mean1 - mean2
    se_diff = np.sqrt(se1**2 + se2**2)
    df = len(data1) + len(data2) - 2
    t_crit = stats.t.ppf(0.975, df)
    ci_lower = mean_diff - t_crit * se_diff
    ci_upper = mean_diff + t_crit * se_diff
    return ci_lower, ci_upper

def bootstrap_ci_paired(data1, data2, n_bootstrap=1000, alpha=0.05):
    differences = data1 - data2
    medians = []
    
    for _ in range(n_bootstrap):
        sample_diff = np.random.choice(differences, size=len(differences), replace=True)
        medians.append(np.median(sample_diff))
    
    ci_lower = np.percentile(medians, 100 * alpha / 2)
    ci_upper = np.percentile(medians, 100 * (1 - alpha / 2))
    return ci_lower, ci_upper

def calculate_paired_statistics(data1, data2, force_to_ttest = False, force_to_wilcoxon = False):
    # Check if the differences are normally distributed
    differences = data1 - data2
    _, p_value_normality = stats.shapiro(differences)

    # If the p-value from the Shapiro-Wilk test is less than 0.05, use the Wilcoxon signed-rank test
    if  force_to_wilcoxon is True and force_to_ttest is False:
        test_stat, p_value = stats.wilcoxon(data1, data2)
        test_used = "Wilcoxon signed-rank"
        ci_lower, ci_upper = bootstrap_ci_paired(data1, data2)
   
    elif (force_to_ttest is False and force_to_wilcoxon is False) and p_value_normality < 0.05:   
        test_stat, p_value = stats.wilcoxon(data1, data2)
        test_used = "Wilcoxon signed-rank"
        ci_lower, ci_upper = bootstrap_ci_paired(data1, data2)
        
    elif p_value_normality >= 0.05  or (force_to_ttest is True and force_to_wilcoxon is False ):
        test_stat, p_value = stats.ttest_rel(data1, data2)
        test_used = "Paired t-test"
        
        # Calculate the 95% confidence interval for the difference in means
        mean_diff = np.mean(differences)
        se_diff = stats.sem(differences)
        df = len(differences) - 1
        t_crit = stats.t.ppf(0.975, df)
        ci_lower = mean_diff - t_crit * se_diff
        ci_upper = mean_diff + t_crit * se_diff
    else:
        raise ValueError('Something is wrong inside calculate_paired_statistics')
    
    
    return {
        'test_used': test_used,
        'p_value': p_value,
        'ci_mean_difference': (ci_lower, ci_upper)
    }, p_value

def get_significance_symbol(p_value: float, significance_level: float) -> str:
    if p_value < 0.001:
        return '***'
    elif p_value < 0.01:
        return '**'
    elif p_value < significance_level:
        return '*'
    else:
        return ''
    

def perform_chi_square(data: pd.DataFrame, col1: str, col2: str, significance_level: float) -> float:
    contingency_table = pd.crosstab(data[col1], data[col2])
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    return p_value


### add_subplot_topleft_label

In [ ]:
from typing import List, Dict, Union
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.transforms import ScaledTranslation
from matplotlib.patches import Rectangle
from scipy.stats import wilcoxon, ttest_rel, sem
import itertools
from scipy import stats
import statsmodels.stats.api as sms

def add_subplot_topleft_label(ax: plt.Axes, label: str, label_params: dict = None) -> None:
    """
    Add a subplot label to the top left corner of a single subplot.
    The label is drawn with a rectangle behind it, with padding both inside and outside the rectangle.

    Args:
        ax (plt.Axes): The subplot Axes object
        label (str): The label text to be added
        label_params (dict): Optional parameters for the label and rectangle
            - 'fontsize': The font size of the label (default: 'medium')
            - 'fontfamily': The font family of the label (default: 'serif')
            - 'rect_color': The background color of the rectangle (default: 'white')
            - 'rect_alpha': The opacity of the rectangle (default: 0.8)
            - 'rect_pad_inner': The padding inside the rectangle (default: 0.3)
            - 'rect_pad_outer': The padding outside the rectangle (default: 0.3)
            - 'rect_edgecolor': The color of the rectangle's edge (default: 'black')
            - 'rect_linewidth': The width of the rectangle's edge (default: 2)
    """
    default_params = {
        'fontsize': 'medium',
        'fontfamily': 'serif',
        'rect_color': 'white',
        'rect_alpha': 0.8,
        'rect_pad_inner': 0.3,
        'rect_pad_outer': 0.3,
        'rect_edgecolor': 'black',
        'rect_linewidth': 2,
        'modifiable_location_of_box_x' : -0.1,
        'modifiable_location_of_box_y': +1.1,
        'modifiable_location_of_box_emtpyspace_x': 1.1,
        'modifiable_location_of_box_emtpyspace_y':.1,
    }
    
    if label_params:
        default_params.update(label_params)
        
    modifable_location_value_x = default_params['modifiable_location_of_box_x']
    modifable_location_value_y = default_params['modifiable_location_of_box_y']
    modifiable_location_of_box_emtpyspace_x=default_params['modifiable_location_of_box_emtpyspace_x']
    modifiable_location_of_box_emtpyspace_y=default_params['modifiable_location_of_box_emtpyspace_y']
    # Get the size of the label text
    fake_text_to_add_figure_size = ax.text(
        modifable_location_value_x*modifiable_location_of_box_emtpyspace_x, modifable_location_value_y*modifiable_location_of_box_emtpyspace_y, " ", transform=(
            ax.transAxes + ScaledTranslation(-7/72, +7/72, ax.figure.dpi_scale_trans)),
    ) 
    label_text = ax.text(
        modifable_location_value_x, modifable_location_value_y, label, transform=(
            ax.transAxes + ScaledTranslation(-7/72, +7/72, ax.figure.dpi_scale_trans)),
        fontsize=default_params['fontsize'], va='bottom', fontfamily=default_params['fontfamily'],
        bbox={
            'boxstyle':f'square,pad={default_params['rect_pad_inner']}',
            'facecolor': default_params['rect_color'],
            'edgecolor': default_params['rect_edgecolor'],
            'linewidth': default_params['rect_linewidth'],
            'alpha': default_params['rect_alpha'],
            'pad': default_params['rect_pad_outer']
        }
    ) 


### plot_stratified_boxplots_auto

In [ ]:
from typing import List, Dict, Union
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.transforms import ScaledTranslation
from matplotlib.patches import Rectangle
from scipy.stats import wilcoxon, ttest_rel, sem
import itertools
from scipy import stats
import statsmodels.stats.api as sms

        
def plot_stratified_boxplots_auto(data, column_sets, color_palete, name_set, unit_set, cutoff_set, xticks, 
                             statistical_test='auto', show_signifants_only=False,
                             list_of_subplot_label: List[str] = None, custom_label_params: Dict[str, Union[str, float, int]] = None):
    """ 
    Plot stratified boxplots with significance testing.

    Parameters:
    data: DataFrame
        The data to be used for plotting.
    column_sets: list of lists
        List of column sets for different categories.
    name_set: list
        List of names for each set of columns.
    unit_set: list
        List of units for each set of columns.
    cutoff_set: list
        List of cutoff values for each set of columns.
    xticks: list
        List of xtick labels for the boxplots.
    statistical_test: str, optional
        The statistical test to be used ('wilcoxon' or 'ttest'). Default is 'wilcoxon'.
    list_of_subplot_label: list
        List of subplot labels (e.g., ['a', 'b', 'c', ...]).
        
    Returns:
    fig: Figure
        The matplotlib figure object.
    """
        
    # Define the statistical test function
    if statistical_test in ('wilcoxon', 'ttest', 'auto'):
        test_func = calculate_paired_statistics
    else:
        raise ValueError("statistical_test must be 'wilcoxon' or 'ttest' or 'auto'")

    # Create the plots
    fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(40, 50), sharey=False)
    x_axs = 0
    y_axs = 0
    
    for i, columns in enumerate(column_sets):
        melted_data = pd.melt(data, value_vars=columns, var_name='Timepoint', value_name='Value')

        sns.boxplot(x='Timepoint', y='Value', data=melted_data,
                    ax=axs[x_axs, y_axs],
                    palette=color_palete,
                    linewidth=5)

        axs[x_axs, y_axs].axhline(y=cutoff_set[i], linestyle=':', color='grey', linewidth=7)

        sns.stripplot(x='Timepoint', y='Value', data=melted_data,
                      ax=axs[x_axs, y_axs],
                      palette=["black"],
                      size=13,
                      alpha=0.5)
        axs[x_axs, y_axs].set_title(f'{name_set[i]}', fontsize=55)
        axs[x_axs, y_axs].set_xlabel(' ', fontsize=35)
        axs[x_axs, y_axs].set_ylabel(f'{unit_set[i]}', fontsize=35)
        axs[x_axs, y_axs].tick_params(axis='both', labelsize=30)
        axs[x_axs, y_axs].set_xticklabels(xticks)
        

            
        bottom, top = axs[x_axs, y_axs].get_ylim()
        y_range = top - bottom

        combinations = itertools.combinations(columns, 2)
        significant_combinations = []
        for combination in combinations:
            # Extract data for the two time points
            valid_comb_data = data.dropna(subset=[combination[0], combination[1]])
            tp1 = valid_comb_data[combination[0]]
            tp2 = valid_comb_data[combination[1]]
            if statistical_test == 'ttest':
                stat, p_value = test_func(tp1, tp2, force_to_ttest=True)
            elif statistical_test == 'wilcoxon':
                stat, p_value = test_func(tp1, tp2, force_to_wilcoxon=True)
            else: 
                stat, p_value= test_func(tp1, tp2)
                
            if statistical_test =='auto':
                print(f"The statistic test for columns of '{columns[0]}' and combination of {combination}: \n {stat}")
            ci_low, ci_high = stat['ci_mean_difference']
            significant_combinations.append([combination, p_value, (ci_low, ci_high)])

        for j, significant_combination in enumerate(significant_combinations):
            x1 = columns.index(significant_combination[0][0])
            x2 = columns.index(significant_combination[0][1])
            p = significant_combination[1]
            
            if show_signifants_only is True and p >=0.05:
                continue
                
            
            
            level = len(significant_combinations) - j
            bar_height = (y_range * 0.1 * level) + top
            bar_tips = bar_height - (y_range * 0.02)
            axs[x_axs, y_axs].plot(
                [x1, x1, x2, x2],
                [bar_tips, bar_height, bar_height, bar_tips], lw=1, c='k'
            )

            p = significant_combination[1]
            sig_symbol= f"{p:.3f}"
            #sig_symbol = get_significance_symbol(p, 0.05)

            CI95 = significant_combination[2]
            text4plot = f"95CI: [{CI95[0]:.2f}, {CI95[1]:.2f}]; P={sig_symbol}"
            text_height = bar_height + (y_range * 0.01)
            if p <0.05:
                axs[x_axs, y_axs].text((x1 + x2) / 2, text_height, text4plot, ha='center', va='bottom', c='k', fontsize=30, weight='bold')
            else:
                axs[x_axs, y_axs].text((x1 + x2) / 2, text_height, text4plot, ha='center', va='bottom', c='k', fontsize=30)
            
        # Annotate the number of participants  
        for timepoint in columns: 
            column_data = data[timepoint].dropna()
            num_participants = len(column_data)
            if num_participants > 0:
                x_pos = columns.index(timepoint)
                #y_pos = np.median(column_data) * 1.1
                #y_pos = max(column_data)
                y_pos = -1 * 0.02 * axs[x_axs, y_axs].get_ylim()[1]
                axs[x_axs, y_axs].text(x_pos, y_pos, f'n={num_participants}', ha='center', va='bottom', fontsize=30, bbox=dict(facecolor='white', alpha=0.6))

        if list_of_subplot_label:
            subplot_label=list_of_subplot_label[i]
            add_subplot_topleft_label(ax = axs[x_axs, y_axs], label=subplot_label, label_params=custom_label_params)

        y_axs += 1
        if y_axs > 1:
            y_axs = 0
            x_axs += 1

    plt.tight_layout()
    plt.show()

    return fig

In [ ]:
# Example usage
data = pd.read_csv(file_path)
data_HD_gold = data[(data["case/control"] == "HD case") & (data["HD Cohort Type"] == "with HD0")]
column_sets = [
    ['IgGPre', 'IgGpost1or2D1', 'IgGpost21D1', 'IgGpost1or2D2', 'IgGpost21D2', 'IgGpost60D2'],
    ['NeutrolizingPre', 'Neutrolizing1or2D1', 'Neutrolizing21D1', 'Neutrolizing1or2D2', 'Neutrolizing21D2', 'Neutrolizing60D2'],
    ['AntiSpikepre', 'AntiSpike1or2D1', 'AntiSpike21D1', 'AntiSpike1or2D2', 'AntiSpike21D2', 'AntiSpike60D2'],
    ['AntiRBDpre', 'AntiRBD1or2D1', 'AntiRBD21D1', 'AntiRBD1or2D2', 'AntiRBD21D2', 'AntiRBD60D2']
]
color_palete = ["#00D5E5", "#00D5E5","#00D5E5","#00D5E5","#00D5E5","#00D5E5",]
name_set = ['IgG', 'Neutralizing Ab', 'Anti Spike Ab', 'Anti RBD']
unit_set = ['Index', 'micg/ml', 'RU/mL', 'Index']

cutoff_set = [1.1, 2.5, 8, 1.1]
xticks = ['Pre', '1or2D1', '21D1', '1or2D2', '21D2', '60D2']

list_of_subplot_label = ['a', 'b', 'c', 'd']
custom_label_params =  {
        'fontsize': '70',
        'fontfamily': 'serif',
        'rect_color': 'white',
        'rect_alpha': 0.8,
        'rect_pad_inner': 0.2,
        'rect_pad_outer': 0.3,
        'rect_edgecolor': 'black',
        'rect_linewidth': 6,
        'modifiable_location_of_box_x':-0.05,
        'modifiable_location_of_box_y':1.01,
        'modifiable_location_of_box_emtpyspace': 1.05,
    }

fig = plot_stratified_boxplots_auto(data_HD_gold, column_sets,color_palete, name_set, unit_set, cutoff_set, xticks, statistical_test='wilcoxon', list_of_subplot_label=list_of_subplot_label, custom_label_params=custom_label_params)

# Save the figure if needed
#save_figure(plot=fig, destination_path="path/to/save", figure_name="Immunogenicity_HD_Gold")

## plot_serial_stackedCount_bar

dorost nashod :) 

In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# from scipy.stats import chi2_contingency
# from typing import List, Dict, Tuple, Optional

# def plot_serial_stackedCount_bar(
#     data: pd.DataFrame,
#     column_sets: List[List[str]],
#     name_set: List[str],
#     xticks: List[str],
#     plot_variables: Dict[str, Dict[str, str]],
#     x_label: str = "Time Point",
#     y_label: str = "Percentage",
#     show_chi_square: bool = True,
#     list_of_subplot_label: List[str] = None,
#     custom_label_params: dict = None,
#     significance_level: float = 0.05,
# ) -> Tuple[plt.Figure, plt.Axes]:
#     """
#     Plots the antibody response over time for different antibody types in Huntington's Disease (HD) patients without previous COVID-19 infection.
#     """
#     # Create the figure and axes
#     fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(40, 18), sharex=False, sharey=True)

#     for i, columns in enumerate(column_sets):
#         # Calculate the counts and percentages
#         counts = _calculate_counts_and_percentages(data, columns)

#         # Determine the subplot location
#         x_axs, y_axs = divmod(i, 2)

#         # Plot the stacked bar chart
#         _plot_stacked_bar(axs[x_axs, y_axs], counts, plot_variables)

#         # Set the plot title, labels, and tick formatting
#         _set_plot_properties(axs[x_axs, y_axs], name_set[i], x_label, y_label, xticks)

#         # Perform the chi-square test between adjacent time points
#         if show_chi_square:
#             _add_chi_square_annotations(axs[x_axs, y_axs], data, columns, significance_level)

#         # Add subplot labels if provided
#         if list_of_subplot_label:
#             _add_subplot_label(axs[x_axs, y_axs], list_of_subplot_label[i], custom_label_params)

#     plt.suptitle(title, fontsize=60)
#     plt.tight_layout()

#     return fig, axs

# def _calculate_counts_and_percentages(data: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
#     """
#     Calculates the counts and percentages for the given columns.
#     """
#     counts = data[columns].apply(pd.Series.value_counts).T
#     counts = counts.fillna(0)
#     counts.index.name = 'Timepoint'
#     counts.reset_index(inplace=True)

#     total_counts = counts['positive'] + counts['negative']
#     counts['positive%'] = (counts['positive'] / total_counts) * 100
#     counts['negative%'] = (counts['negative'] / total_counts) * 100

#     return counts

# def _plot_stacked_bar(ax: plt.Axes, counts: pd.DataFrame, plot_variables: Dict[str, Dict[str, str]]) -> None:
#     """
#     Plots the stacked bar chart for the given data and axes.
#     """
#     bars = counts[['positive%', 'negative%']].plot(kind='bar', stacked=True, ax=ax,
#                                                   color=[plot_variables['positive%']['color'],
#                                                          plot_variables['negative%']['color']])

#     # Add the counts on top of each bar
#     for i, bar in enumerate(bars.containers):
#         for j, b in enumerate(bar):
#             ax.text(b.get_x() + b.get_width() / 2, b.get_height() + 2, 
#                     int(counts.iloc[i][['positive', 'negative'][j]]),
#                     ha='center', va='bottom', fontsize=18)

# def _set_plot_properties(ax: plt.Axes, title: str, x_label: str, y_label: str, xticks: List[str]) -> None:
#     """
#     Sets the plot title, labels, and tick formatting.
#     """
#     ax.set_title(title, fontsize=55)
#     ax.set_xlabel(x_label, fontsize=35)
#     ax.set_ylabel(y_label, fontsize=35)
#     ax.tick_params(axis='both', labelsize=35)
#     ax.set_xticklabels(xticks, rotation=25, ha='right')
#     ax.set_ylim(0, 100)  # Set the y-axis range to 0-100%
#     ax.get_legend().remove()

# def _add_chi_square_annotations(ax: plt.Axes, data: pd.DataFrame, columns: List[str], significance_level: float) -> None:
#     """
#     Adds the chi-square test results as annotations on the plot.
#     """
#     for k in range(len(columns) - 1):
#         p_value = perform_chi_square(data, columns[k], columns[k+1], significance_level)
#         significance = get_significance_symbol(p_value, significance_level)
#         if significance:
#             x1 = k - 0.4  # Adjust the x-position for the first bar
#             x2 = k + 0.4  # Adjust the x-position for the second bar
#             y = 100  # Set the y-position to 100% for the annotation
#             ax.annotate(significance, 
#                         xy=((x1+x2)/2, y),
#                         xytext=(0, 20),
#                         textcoords="offset points",
#                         ha='center',
#                         va='bottom',
#                         fontsize=40,
#                         color='red')

# def _add_subplot_label(ax: plt.Axes, label: str, label_params: dict = None) -> None:
#     """
#     Adds a label to the top-left corner of a subplot.
#     """
#     if label_params is None:
#         label_params = {'fontsize': 70, 'fontfamily': 'serif', 'color': 'black'}
#     ax.text(0.02, 0.98, label, transform=ax.transAxes, va='top', **label_params)

# def perform_chi_square(data: pd.DataFrame, col1: str, col2: str, significance_level: float) -> float:
#     """
#     Performs the chi-square test between the given columns.
#     """
#     contingency_table = pd.crosstab(data[col1], data[col2])
#     chi2, p_value, dof, expected = chi2_contingency(contingency_table)
#     return p_value

# def get_significance_symbol(p_value: float, significance_level: float) -> str:
#     """
#     Determines the significance symbol based on the p-value.
#     """
#     if p_value < 0.001:
#         return '***'
#     elif p_value < 0.01:
#         return '**'
#     elif p_value < significance_level:
#         return '*'
#     else:
#         return ''

In [ ]:
# fig = plot_serial_stackedCount_bar(
#     data=data,
#     column_sets=[
#         ['IgGPrepositive', 'IgG positive 1or2D1', 'IgG positive 21D1', 'IgG positive 1or2D2', 'IgG positive 21D2', 'IgG positve 60D2'],
#         ['NeutrolizingPrePos', 'Neutrolizing1or2D1Pos', 'Neutrolizing21D1Pos', 'Neutrolizing1or2D2Pos', 'Neutrolizing21D2Pos', 'Neutrolizing60D2Pos'],
#         ['AntiSpikeprePOS', 'AntiSpike1or2D1POS', 'AntiSpike21D1POS', 'AntiSpike1or2D2POS', 'AntiSpike21D2POS', 'AntiSpike60D2POS'],
#         ['AntiRBDprePOS', 'AntiRBD1or2D1POS', 'AntiRBD21D1POS', 'AntiRBD1or2D2POS', 'AntiRBD21D2POS', 'AntiRBD60D2POS']
#     ],
#     name_set=['IgG', 'Neutralizing Ab', 'Anti Spike Ab', 'Anti RBD'],
#     xticks=['Pre', '2d Post D1', '21d Post D1', '2d Post D2', '21d Post D2', '60d Post D2'],
#     plot_variables={
#         'positive%': {'color': '#3c93c2'},
#         'negative%': {'color': '#dceef4'}
#     },
#     x_label="Time Point",
#     y_label="Percentage",
#     show_chi_square=True,
#     list_of_subplot_label=['a','b','c','d'],
#     custom_label_params =  {
#         'fontsize': '70',
#         'fontfamily': 'serif',
#         'rect_color': 'white',
#         'rect_alpha': 0.8,
#         'rect_pad_inner': 0.2,
#         'rect_pad_outer': 0.3,
#         'rect_edgecolor': 'black',
#         'rect_linewidth': 6,
#         'modifiable_location_of_box_x':-0.05,
#         'modifiable_location_of_box_y':1.01,
#         'modifiable_location_of_box_emtpyspace': 1.1,
#     }
# )

## plot_comparison_boxplots_auto

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind, mannwhitneyu, sem
import itertools
import numpy as np
from scipy import stats
import statsmodels.stats.api as sms

import numpy as np
from scipy import stats
import statsmodels.stats.api as sms

def bootstrap_ci(data1, data2, n_bootstrap=1000, alpha=0.05):
    combined = np.concatenate([data1, data2])
    n1 = len(data1)
    idx = np.arange(len(combined))
    medians = []

    for _ in range(n_bootstrap):
        np.random.shuffle(idx)
        sample1 = combined[idx[:n1]]
        sample2 = combined[idx[n1:]]
        medians.append(np.median(sample1) - np.median(sample2))
    
    ci_lower = np.percentile(medians, 100 * alpha / 2)
    ci_upper = np.percentile(medians, 100 * (1 - alpha / 2))
    return ci_lower, ci_upper

def calculate_statistics(data1, data2):
    # Check if the data is normally distributed
    _, p1 = stats.shapiro(data1)
    _, p2 = stats.shapiro(data2)

    # Calculate means and standard errors
    mean1 = np.mean(data1)
    mean2 = np.mean(data2)
    se1 = stats.sem(data1)
    se2 = stats.sem(data2)

    # If either of the p-values from the Shapiro-Wilk test are less than 0.05, use Mann-Whitney U test
    if p1 < 0.05 or p2 < 0.05:
        test_stat, p_value = stats.mannwhitneyu(data1, data2)
        test_used = "Mann-Whitney U"
        ci_lower, ci_upper = bootstrap_ci(data1, data2)
    else:
        test_stat, p_value = stats.ttest_ind(data1, data2, equal_var=False)  # Welch's t-test
        test_used = "Independent t-test"
        
        # Calculate the 95% confidence interval for the difference in means
        mean_diff = mean1 - mean2
        se_diff = np.sqrt(se1**2 + se2**2)
        df = len(data1) + len(data2) - 2
        t_crit = stats.t.ppf(0.975, df)
        ci_lower = mean_diff - t_crit * se_diff
        ci_upper = mean_diff + t_crit * se_diff

    return {
        'test_used': test_used,
        'p_value': p_value,
        'ci_mean_difference': (ci_lower, ci_upper)
    }, p_value
    
def calc_95ci_ind(data1, data2):
    mean1 = np.mean(data1)
    mean2 = np.mean(data2)
    se1 = stats.sem(data1)
    se2 = stats.sem(data2)
    mean_diff = mean1 - mean2
    se_diff = np.sqrt(se1**2 + se2**2)
    df = len(data1) + len(data2) - 2
    t_crit = stats.t.ppf(0.975, df)
    ci_lower = mean_diff - t_crit * se_diff
    ci_upper = mean_diff + t_crit * se_diff
    return ci_lower, ci_upper

def plot_comparison_boxplots_auto(data, column_sets, name_set, unit_set, cutoff_set, xticks, 
                             stratification_column, groups_dict, statistical_test='auto',fig_size=(40,40), list_of_subplot_label=None, custom_subplotlabel_params=None):
    """
    Plot comparison boxplots with significance testing between two groups.

    Parameters:
    data: DataFrame
        The data to be used for plotting.
    column_sets: list of lists
        List of column sets for different categories.
    name_set: list
        List of names for each set of columns.
    unit_set: list
        List of units for each set of columns.
    cutoff_set: list
        List of cutoff values for each set of columns.
    xticks: list
        List of xtick labels for the boxplots.
    stratification_column: str
        The column name to define the two groups for comparison.
    groups_dict: dict
        Dictionary of group labels in the data and their desired labels and colors in the figure.
    statistical_test: str, optional
        The statistical test to be used ('ttest' or 'mannwhitney'). Default is 'ttest'.

    Returns:
    fig: Figure
        The matplotlib figure object.
    """
    
    # Define the statistical test function
    if statistical_test == 'ttest':
        test_func = ttest_ind
    elif statistical_test == 'mannwhitney':
        test_func = mannwhitneyu
    elif statistical_test == 'auto':
        test_func = calculate_statistics
    else:
        raise ValueError("statistical_test must be 'ttest' or 'mannwhitney'")

    # Filter data to include only specified groups
    valid_groups = list(groups_dict.keys())
    filtered_data = data[data[stratification_column].isin(valid_groups)].copy()

    # Map group labels and colors for the plot
    filtered_data['Group'] = filtered_data[stratification_column].map(lambda x: groups_dict[x]['label in figure'])
    colors = [groups_dict[group]['color of boxplot'] for group in valid_groups]

    # Create the plots
    fig, axs = plt.subplots(nrows=2, ncols=2, figsize=fig_size, sharey=False)
    x_axs = 0
    y_axs = 0

    for i, columns in enumerate(column_sets):
        melted_data = pd.melt(filtered_data, id_vars=['Group'], value_vars=columns, var_name='Timepoint', value_name='Value')

        sns.boxplot(x='Timepoint', y='Value', hue='Group', data=melted_data,
                    ax=axs[x_axs, y_axs],
                    palette=colors,
                    linewidth=5,
                    gap=.2)

        axs[x_axs, y_axs].axhline(y=cutoff_set[i], linestyle=':', color='grey', linewidth=7)

        sns.stripplot(x='Timepoint', y='Value', hue='Group', data=melted_data,
                      ax=axs[x_axs, y_axs],
                      palette=["black"] * len(valid_groups),
                      size=13,
                      alpha=0.5,
                      dodge=True)

        axs[x_axs, y_axs].set_title(f'{name_set[i]}', fontsize=55,bbox=dict(facecolor='none', edgecolor='none', boxstyle='round,pad=0.5'))
        axs[x_axs, y_axs].set_xlabel(' ', fontsize=35)
        axs[x_axs, y_axs].set_ylabel(f'{unit_set[i]}', fontsize=35)
        axs[x_axs, y_axs].tick_params(axis='both', labelsize=30)
        axs[x_axs, y_axs].set_xticklabels(xticks)

        bottom, top = axs[x_axs, y_axs].get_ylim()
        y_range = top - bottom

        significant_combinations = []
        for timepoint in columns:
            group_data = []
            for group in valid_groups:
                group_data.append(filtered_data[filtered_data[stratification_column] == group][timepoint].dropna())
            if len(group_data[0]) > 0 and len(group_data[1]) > 0:  # Ensure non-zero size for both groups
                stat, p_value = test_func(group_data[0], group_data[1])
                calculate_statistics(group_data[0], group_data[1])
                if statistical_test == 'ttest':
                    ci_low, ci_high = calc_95ci_ind(group_data[0], group_data[1])
                elif statistical_test == 'mannwhitney':
                    ci_low, ci_high = bootstrap_ci(group_data[0], group_data[1])
                elif statistical_test == 'auto':
                    print(f"Test used for dateset: {columns} and timepoint: {timepoint}: \n {stat['test_used']}")
                    print(stat)
                    ci_low, ci_high = stat['ci_mean_difference']
                else:
                    raise ValueError("The statiscial_test is not supported. Chose 'ttest' or 'mannwhitney'. ")
                significant_combinations.append([timepoint, p_value, (ci_low, ci_high), "Valid_stat_2show"])
            else:
                significant_combinations.append([timepoint, np.nan, (np.nan, np.nan),"Invalid_stat_2show"])

        for j, significant_combination in enumerate(significant_combinations):
            if significant_combination[3] == "Valid_stat_2show": # then continue if mean differnece and p valu is valid for this pair
                x1 = columns.index(significant_combination[0])
                x2 = x1 + 0.1
                level = len(significant_combinations) - j
                bar_height = (y_range * 0.1 * level) + top
                bar_tips = bar_height - (y_range * 0.02)
                axs[x_axs, y_axs].plot(
                    [x1, x1, x2, x2],
                    [bar_tips, bar_height, bar_height, bar_tips], lw=1, c='k'
                )

                p = significant_combination[1]
                sig_symbol= f"{p:.3f}"
                #if p < 0.001:
                #    sig_symbol = '***'
                #elif p < 0.01:
                #    sig_symbol = '**'
                #elif p < 0.05:
                #    sig_symbol = '*'
                #else:
                #    sig_symbol = 'NS'

                stat_CI95 = significant_combination[2]
                if np.isnan(stat_CI95[0]):
                    text4plot = f"{sig_symbol}"
                else:
                    text4plot = f"95CI: [{stat_CI95[0]:.2f}, {stat_CI95[1]:.2f}]; P={sig_symbol}"
                text_height = bar_height + (y_range * 0.01)
                
                if p<0.05:
                    axs[x_axs, y_axs].text((x1 + x2) / 2, text_height, text4plot, ha='center', va='bottom', c='k', fontsize=30, bbox=dict(facecolor='none', edgecolor='none', boxstyle='round,pad=0.5'), weight='bold')
                else: 
                    axs[x_axs, y_axs].text((x1 + x2) / 2, text_height, text4plot, ha='center', va='bottom', c='k', fontsize=30, bbox=dict(facecolor='none', edgecolor='none', boxstyle='round,pad=0.5'))
        
        # Add number of participants above each box plot
        for timepoint in columns:
            for group in valid_groups:
                group_data = filtered_data[filtered_data[stratification_column] == group][timepoint].dropna()
                num_participants = len(group_data)
                if num_participants > 0:
                    x_pos = columns.index(timepoint) + (0.2 if group == valid_groups[1] else -0.2)
                    y_pos = max(group_data) *0.9
                    axs[x_axs, y_axs].text(x_pos, y_pos, f'n={num_participants}', ha='center', va='bottom', fontsize=25, bbox=dict(facecolor='white', alpha=0.6))
                    
        if list_of_subplot_label:
            subplot_label=list_of_subplot_label[i]
            add_subplot_topleft_label(ax=axs[x_axs, y_axs], label=subplot_label, label_params=custom_subplotlabel_params)    
            
        x_axs += 1
        if x_axs > 1:
            x_axs = 0
            y_axs += 1

        
    plt.tight_layout()
    plt.show()

    return fig

# Example usage
data = pd.read_csv(file_path)
data_HD_gold = data[(data["case/control"] == "HD case")]
column_sets = [
    ['IgGPre', 'IgGpost1or2D1', 'IgGpost21D1', 'IgGpost1or2D2', 'IgGpost21D2', 'IgGpost60D2'],
    ['NeutrolizingPre', 'Neutrolizing1or2D1', 'Neutrolizing21D1', 'Neutrolizing1or2D2', 'Neutrolizing21D2', 'Neutrolizing60D2'],
    ['AntiSpikepre', 'AntiSpike1or2D1', 'AntiSpike21D1', 'AntiSpike1or2D2', 'AntiSpike21D2', 'AntiSpike60D2'],
    ['AntiRBDpre', 'AntiRBD1or2D1', 'AntiRBD21D1', 'AntiRBD1or2D2', 'AntiRBD21D2', 'AntiRBD60D2']
]
name_set = ['IgG', 'Neutralizing Ab', 'Anti Spike Ab', 'Anti RBD']
unit_set = ['Index', 'micg/ml', 'RU/mL', 'Index']
cutoff_set = [1.1, 2.5, 8, 1.1]
xticks = ['Pre', '1or2D1', '21D1', '1or2D2', '21D2', '60D2']
stratification_column="HD Cohort Type"
groups_dict = {
    "with HD0": {"label in figure": "HD0", "color of boxplot": "#008Dff"},
    "without HD0": {"label in figure": "HD1", "color of boxplot": "#003a7d"}
}
statistical_test= "mannwhitney" #"auto" OR "ttest" OR "auto"

fig_size=(40,40)

list_of_subplot_label=['a','b','c','d']

custom_subplotlabel_params={
        'fontsize': '70',
        'fontfamily': 'serif',
        'rect_color': 'white',
        'rect_alpha': 0.8,
        'rect_pad_inner': 0.3,
        'rect_pad_outer': 0.3,
        'rect_edgecolor': 'black',
        'rect_linewidth': 4,
        'modifiable_location_of_box_x' : -0.1,
        'modifiable_location_of_box_y': +1.0,
        'modifiable_location_of_box_emtpyspace_x': 1.3,
        'modifiable_location_of_box_emtpyspace_y':1.08,
    }
fig = plot_comparison_boxplots_auto(data_HD_gold, column_sets, name_set, unit_set, cutoff_set, xticks, 
                            stratification_column=stratification_column, groups_dict=groups_dict, statistical_test=statistical_test, fig_size=fig_size, list_of_subplot_label=list_of_subplot_label,custom_subplotlabel_params=custom_subplotlabel_params )



# Basi Charectristics

Code in Data Analysis

# COVID-19 Breakthrough Infection 

Code in Data Analysis

_________
# Immunogenicity

## Fig 3a: Immunogenicity for all HD 

In [ ]:
# ALL HDs + excluding previous COVID-19 
data = pd.read_csv(file_path)
data_HD_NoPreviousCOVID =  data[(data["case/control"] == "HD case") & (data["History of COVID-19 infection"] == "No")]
#data_HD_gold = data[(data["case/control"] == "HD case") & (data["HD Cohort Type"] == "with HD0")]

column_sets = [
    ['IgGPre', 'IgGpost1or2D1', 'IgGpost21D1', 'IgGpost1or2D2', 'IgGpost21D2', 'IgGpost60D2'],
    ['NeutrolizingPre', 'Neutrolizing1or2D1', 'Neutrolizing21D1', 'Neutrolizing1or2D2', 'Neutrolizing21D2', 'Neutrolizing60D2'],
    ['AntiSpikepre', 'AntiSpike1or2D1', 'AntiSpike21D1', 'AntiSpike1or2D2', 'AntiSpike21D2', 'AntiSpike60D2'],
    ['AntiRBDpre', 'AntiRBD1or2D1', 'AntiRBD21D1', 'AntiRBD1or2D2', 'AntiRBD21D2', 'AntiRBD60D2']
]
# config inside set
color_palete = ["#00D5E5", "#00D5E5", "#3C93C2", "#00D5E5", "#3C93C2", "#3C93C2"]
xticks = ['Pre', '1or2D1', '21D1', '1or2D2', '21D2', '60D2']
# config of set - stat
statistical_test = 'wilcoxon' # 'auto' 'wilcoxon' 'ttest'
# config of set
name_set = ['IgG', 'Neutralizing Ab', 'Anti Spike Ab', 'Anti RBD']
unit_set = ['Index', 'micg/ml', 'RU/mL', 'Index']
cutoff_set = [1.1, 2.5, 8, 1.1]
# config of set - labels
list_of_subplot_label = ['a', 'b', 'c', 'd']
custom_label_params =  {
        'fontsize': '70',
        'fontfamily': 'serif',
        'rect_color': 'white',
        'rect_alpha': 0.8,
        'rect_pad_inner': 0.2,
        'rect_pad_outer': 0.3,
        'rect_edgecolor': 'black',
        'rect_linewidth': 6,
        'modifiable_location_of_box_x':-0.05,
        'modifiable_location_of_box_y':1,
        'modifiable_location_of_box_emtpyspace_x': 1.05,
        'modifiable_location_of_box_emtpyspace_y': 1.05,
    }

fig = plot_stratified_boxplots_auto(data_HD_NoPreviousCOVID, column_sets,color_palete, name_set, unit_set, cutoff_set, xticks, statistical_test='wilcoxon', list_of_subplot_label=list_of_subplot_label, custom_label_params=custom_label_params)

# Save the figure if needed
save_figure(plot=fig, destination_path=figure_destination_path, figure_name="Figure 3a")


## Fig 3b: Immunogenicity for all HD 

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency

# ALL HDs + excluding previous COVID-19 
data = pd.read_csv(file_path)  # Make sure 'file_path' is defined
data_HD_NoPreviousCOVID = data[(data["case/control"] == "HD case") & (data["History of COVID-19 infection"] == "No")]

column_sets = [
    ['IgGPrepositive', 'IgG positive 1or2D1', 'IgG positive 21D1', 'IgG positive 1or2D2', 'IgG positive 21D2', 'IgG positve 60D2'],
    ['NeutrolizingPrePos', 'Neutrolizing1or2D1Pos', 'Neutrolizing21D1Pos', 'Neutrolizing1or2D2Pos', 'Neutrolizing21D2Pos', 'Neutrolizing60D2Pos'],    
    ['AntiSpikeprePOS', 'AntiSpike1or2D1POS', 'AntiSpike21D1POS', 'AntiSpike1or2D2POS', 'AntiSpike21D2POS', 'AntiSpike60D2POS'],
    ['AntiRBDprePOS', 'AntiRBD1or2D1POS', 'AntiRBD21D1POS', 'AntiRBD1or2D2POS', 'AntiRBD21D2POS', 'AntiRBD60D2POS']
]
name_set = ['IgG', 'Neutralizing Ab', 'Anti Spike Ab',  'Anti RBD']


def perform_chi_square(data, col1, col2):
    contingency_table = pd.crosstab(data[col1], data[col2])
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    return p_value




fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(40, 20), sharey=False)
plt.subplots_adjust(hspace=0.3, wspace=100)

for i, columns in enumerate(column_sets):
    counts = data_HD_NoPreviousCOVID[columns].apply(pd.Series.value_counts).T
    counts = counts.fillna(0)
    counts.index.name = 'Timepoint'
    counts.reset_index(inplace=True)
    
    counts['positive%'] = counts['positive'] / (counts['positive'] + counts['negative']) * 100
    counts['negative%'] = counts['negative'] / (counts['positive'] + counts['negative']) * 100
    countspercent = counts.drop(columns=['positive', 'negative'])

    x_axs, y_axs = divmod(i, 2)

    countspercent.plot(kind='bar', stacked=True, 
                ax=axs[x_axs, y_axs],
                color=['#3c93c2', '#CDCDCD'])

    axs[x_axs, y_axs].set_title(f'{name_set[i]}', fontsize=55)
    axs[x_axs, y_axs].set_xlabel(' ', fontsize=35)
    axs[x_axs, y_axs].set_ylabel('Percentage', fontsize=35)
    axs[x_axs, y_axs].tick_params(axis='both', labelsize=35)
    axs[x_axs, y_axs].set_xticklabels(['Pre', '2d Post D1', '21d Post D1', '2d Post D2', '21d Post D2', '60d Post D2'], rotation=25, ha='right')
    axs[x_axs, y_axs].get_legend().remove()
    
    for j, patch in enumerate(axs[x_axs, y_axs].containers):
        for k, rect in enumerate(patch.patches):
            height = rect.get_height()
            x = rect.get_x() + rect.get_width() / 2
            y = rect.get_y() + height / 2
            
            # Get the count value
            count = int(counts.iloc[k]['positive'] if j == 0 else counts.iloc[k]['negative'])
            if count != 0:
                axs[x_axs, y_axs].annotate(f"{count}",
                            (x, y),
                            xytext=(0, 0),
                            textcoords="offset points",
                            ha='center',
                            va='center',
                            fontsize=30,
                            color='black')
    
    # Perform chi-square test between adjacent time points
    for k in range(len(columns) - 1):
        p_value = perform_chi_square(data_HD_NoPreviousCOVID, columns[k], columns[k+1])
        significance = ''
        if p_value < 0.001:
            significance = '***'
        elif p_value < 0.01:
            significance = '**'
        elif p_value < 0.05:
            significance = '*'
        elif p_value>= 0.05:
            significance = 'ns'
        
        if significance:
            x1 = k
            x2 = k + 1
            y = axs[x_axs, y_axs].get_ylim()[1] * 1.05
            axs[x_axs, y_axs].annotate(significance, 
                                       xy=((x1+x2)/2, y),
                                       xytext=(0, 0),
                                       textcoords="offset points",
                                       ha='center',
                                       va='center',
                                       fontsize=40,
                                       color='red')
            
    list_of_subplot_label=['e','f','g','h']
    label_params = {
    'fontsize': '70',
    'fontfamily': 'serif',
    'rect_color': 'white',
    'rect_alpha': 0.8,
    'rect_pad_inner': 0.3,
    'rect_pad_outer': 0.3,
    'rect_edgecolor': 'black',
    'rect_linewidth': 4,
    'modifiable_location_of_box_x' : -0.1,
    'modifiable_location_of_box_y': +1.1,
    'modifiable_location_of_box_emtpyspace_x': 1.4,
    'modifiable_location_of_box_emtpyspace_y': 1.2,
    }
    if list_of_subplot_label:
        subplot_label=list_of_subplot_label[i]
        add_subplot_topleft_label(ax= axs[x_axs, y_axs],label=subplot_label,label_params=label_params)



plt.tight_layout()
plt.show()

# Save the figure if needed
save_figure(plot=fig, destination_path=figure_destination_path, figure_name="Figure 3b")

## Supp Fig S2a: Immunogenic stratified by HD0 and HD1

In [ ]:
# Example usage
data = pd.read_csv(file_path)
data_HD_NoPreviousCOVID =  data[(data["case/control"] == "HD case") & (data["History of COVID-19 infection"] == "No")]

column_sets = [
    ['IgGPre', 'IgGpost1or2D1', 'IgGpost21D1', 'IgGpost1or2D2', 'IgGpost21D2', 'IgGpost60D2'],
    ['NeutrolizingPre', 'Neutrolizing1or2D1', 'Neutrolizing21D1', 'Neutrolizing1or2D2', 'Neutrolizing21D2', 'Neutrolizing60D2'],
    ['AntiSpikepre', 'AntiSpike1or2D1', 'AntiSpike21D1', 'AntiSpike1or2D2', 'AntiSpike21D2', 'AntiSpike60D2'],
    ['AntiRBDpre', 'AntiRBD1or2D1', 'AntiRBD21D1', 'AntiRBD1or2D2', 'AntiRBD21D2', 'AntiRBD60D2']
]
name_set = ['IgG', 'Neutralizing Ab', 'Anti Spike Ab', 'Anti RBD']
unit_set = ['Index', 'micg/ml', 'RU/mL', 'Index']
cutoff_set = [1.1, 2.5, 8, 1.1]
xticks = ['Pre', '1or2D1', '21D1', '1or2D2', '21D2', '60D2']

stratification_column="HD Cohort Type"
groups_dict = {
    "with HD0": {"label in figure": "HD0", "color of boxplot": "#00D5E5"},
    "without HD0": {"label in figure": "HD1", "color of boxplot": "#3C93C2"}
}

statistical_test= "mannwhitney" #"mannwhitney" OR "ttest"
fig_size=(40,40)


fig_size=(40,40)

list_of_subplot_label=['a','b','c','d']

custom_subplotlabel_params={
        'fontsize': '70',
        'fontfamily': 'serif',
        'rect_color': 'white',
        'rect_alpha': 0.8,
        'rect_pad_inner': 0.3,
        'rect_pad_outer': 0.3,
        'rect_edgecolor': 'black',
        'rect_linewidth': 4,
        'modifiable_location_of_box_x' : -0.1,
        'modifiable_location_of_box_y': +1.0,
        'modifiable_location_of_box_emtpyspace_x': 1.3,
        'modifiable_location_of_box_emtpyspace_y':1.08,
    }

fig = plot_comparison_boxplots_auto(data_HD_NoPreviousCOVID, column_sets, name_set, unit_set, cutoff_set, xticks, 
                               stratification_column=stratification_column, groups_dict=groups_dict, statistical_test=statistical_test, fig_size=fig_size, list_of_subplot_label=list_of_subplot_label, custom_subplotlabel_params=custom_subplotlabel_params)
save_figure(plot=fig, destination_path=figure_destination_path, figure_name="Supplemantary Figure S2")

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency

# ALL HDs + excluding previous COVID-19 
data = pd.read_csv(file_path)  # Make sure 'file_path' is defined
data_HD_NoPreviousCOVID = data[(data["case/control"] == "HD case") & (data["History of COVID-19 infection"] == "No")]

column_sets = [
    ['IgGPrepositive', 'IgG positive 1or2D1', 'IgG positive 21D1', 'IgG positive 1or2D2', 'IgG positive 21D2', 'IgG positve 60D2'],
    ['NeutrolizingPrePos', 'Neutrolizing1or2D1Pos', 'Neutrolizing21D1Pos', 'Neutrolizing1or2D2Pos', 'Neutrolizing21D2Pos', 'Neutrolizing60D2Pos'],    
    ['AntiSpikeprePOS', 'AntiSpike1or2D1POS', 'AntiSpike21D1POS', 'AntiSpike1or2D2POS', 'AntiSpike21D2POS', 'AntiSpike60D2POS'],
    ['AntiRBDprePOS', 'AntiRBD1or2D1POS', 'AntiRBD21D1POS', 'AntiRBD1or2D2POS', 'AntiRBD21D2POS', 'AntiRBD60D2POS']
]
name_set = ['IgG', 'Neutralizing Ab', 'Anti Spike Ab',  'Anti RBD']


def perform_chi_square(data, col1, col2):
    contingency_table = pd.crosstab(data[col1], data[col2])
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    return p_value




fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(40, 20), sharey=False)
plt.subplots_adjust(hspace=0.3, wspace=100)

for i, columns in enumerate(column_sets):
    counts = data_HD_NoPreviousCOVID[columns].apply(pd.Series.value_counts).T
    counts = counts.fillna(0)
    counts.index.name = 'Timepoint'
    counts.reset_index(inplace=True)
    
    counts['positive%'] = counts['positive'] / (counts['positive'] + counts['negative']) * 100
    counts['negative%'] = counts['negative'] / (counts['positive'] + counts['negative']) * 100
    countspercent = counts.drop(columns=['positive', 'negative'])

    x_axs, y_axs = divmod(i, 2)

    countspercent.plot(kind='bar', stacked=True, 
                ax=axs[x_axs, y_axs],
                color=['#3c93c2', '#CDCDCD'])

    axs[x_axs, y_axs].set_title(f'{name_set[i]}', fontsize=55)
    axs[x_axs, y_axs].set_xlabel(' ', fontsize=35)
    axs[x_axs, y_axs].set_ylabel('Percentage', fontsize=35)
    axs[x_axs, y_axs].tick_params(axis='both', labelsize=35)
    axs[x_axs, y_axs].set_xticklabels(['Pre', '2d Post D1', '21d Post D1', '2d Post D2', '21d Post D2', '60d Post D2'], rotation=25, ha='right')
    axs[x_axs, y_axs].get_legend().remove()
    
    for j, patch in enumerate(axs[x_axs, y_axs].containers):
        for k, rect in enumerate(patch.patches):
            height = rect.get_height()
            x = rect.get_x() + rect.get_width() / 2
            y = rect.get_y() + height / 2
            
            # Get the count value
            count = int(counts.iloc[k]['positive'] if j == 0 else counts.iloc[k]['negative'])
            if count != 0:
                axs[x_axs, y_axs].annotate(f"{count}",
                            (x, y),
                            xytext=(0, 0),
                            textcoords="offset points",
                            ha='center',
                            va='center',
                            fontsize=30,
                            color='black')
    
    # Perform chi-square test between adjacent time points
    for k in range(len(columns) - 1):
        p_value = perform_chi_square(data_HD_NoPreviousCOVID, columns[k], columns[k+1])
        significance = ''
        if p_value < 0.001:
            significance = '***'
        elif p_value < 0.01:
            significance = '**'
        elif p_value < 0.05:
            significance = '*'
        elif p_value>= 0.05:
            significance = 'ns'
        
        if significance:
            x1 = k
            x2 = k + 1
            y = axs[x_axs, y_axs].get_ylim()[1] * 1.05
            axs[x_axs, y_axs].annotate(significance, 
                                       xy=((x1+x2)/2, y),
                                       xytext=(0, 0),
                                       textcoords="offset points",
                                       ha='center',
                                       va='center',
                                       fontsize=40,
                                       color='red')
            
    list_of_subplot_label=['e','f','g','h']
    label_params = {
    'fontsize': '70',
    'fontfamily': 'serif',
    'rect_color': 'white',
    'rect_alpha': 0.8,
    'rect_pad_inner': 0.3,
    'rect_pad_outer': 0.3,
    'rect_edgecolor': 'black',
    'rect_linewidth': 4,
    'modifiable_location_of_box_x' : -0.1,
    'modifiable_location_of_box_y': +1.1,
    'modifiable_location_of_box_emtpyspace_x': 1.4,
    'modifiable_location_of_box_emtpyspace_y': 1.2,
    }
    if list_of_subplot_label:
        subplot_label=list_of_subplot_label[i]
        add_subplot_topleft_label(ax= axs[x_axs, y_axs],label=subplot_label,label_params=label_params)



plt.tight_layout()
plt.show()

# Save the figure if needed
save_figure(plot=fig, destination_path=figure_destination_path, figure_name="Figure 3b")

## Fig 4a: Immunogenicity : C19 vs no-C19

In [ ]:
# All HD {no previous COVID vs previous COVID}
data = pd.read_csv(file_path)
data_HD = data[(data["case/control"] == "HD case")]

column_sets = [
    ['IgGPre', 'IgGpost1or2D1', 'IgGpost21D1', 'IgGpost1or2D2', 'IgGpost21D2', 'IgGpost60D2'],
    ['NeutrolizingPre', 'Neutrolizing1or2D1', 'Neutrolizing21D1', 'Neutrolizing1or2D2', 'Neutrolizing21D2', 'Neutrolizing60D2'],
    ['AntiSpikepre', 'AntiSpike1or2D1', 'AntiSpike21D1', 'AntiSpike1or2D2', 'AntiSpike21D2', 'AntiSpike60D2'],
    ['AntiRBDpre', 'AntiRBD1or2D1', 'AntiRBD21D1', 'AntiRBD1or2D2', 'AntiRBD21D2', 'AntiRBD60D2']
]
name_set = ['IgG', 'Neutralizing Ab', 'Anti Spike Ab', 'Anti RBD']
unit_set = ['Index', 'micg/ml', 'RU/mL', 'Index']
cutoff_set = [1.1, 2.5, 8, 1.1]
xticks = ['Pre', '1or2D1', '21D1', '1or2D2', '21D2', '60D2']

stratification_column="History of COVID-19 infection"
groups_dict = {
    "No": {"label in figure": "No Exposure (HDs)", "color of boxplot": "#FF1F5b"},
    "Yes": {"label in figure": "Previous COVID-19 (HDs)", "color of boxplot": "#00D5E5"},
}

statistical_test= "auto" #"mannwhitney" OR "ttest"
fig_size=(40,40)
fig = plot_comparison_boxplots_auto(data_HD_gold, column_sets, name_set, unit_set, cutoff_set, xticks, 
                               stratification_column=stratification_column, groups_dict=groups_dict, statistical_test=statistical_test, fig_size=fig_size)
save_figure(plot=fig, destination_path=figure_destination_path, figure_name="Figure 4a")

## Fig 4b: Immunogenicity : C19 vs no-C19

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency

# Load data
data = pd.read_csv(file_path)
data_HD = data[(data["case/control"] == "HD case")]

column_sets = [
    ['IgGPrepositive', 'IgG positive 1or2D1', 'IgG positive 21D1', 'IgG positive 1or2D2', 'IgG positive 21D2', 'IgG positve 60D2'],
    ['NeutrolizingPrePos', 'Neutrolizing1or2D1Pos', 'Neutrolizing21D1Pos', 'Neutrolizing1or2D2Pos', 'Neutrolizing21D2Pos', 'Neutrolizing60D2Pos'],    
    ['AntiSpikeprePOS', 'AntiSpike1or2D1POS', 'AntiSpike21D1POS', 'AntiSpike1or2D2POS', 'AntiSpike21D2POS', 'AntiSpike60D2POS'],
    ['AntiRBDprePOS', 'AntiRBD1or2D1POS', 'AntiRBD21D1POS', 'AntiRBD1or2D2POS', 'AntiRBD21D2POS', 'AntiRBD60D2POS']
]
name_set = ['IgG', 'Neutralizing Ab', 'Anti Spike Ab',  'Anti RBD']
stratification_column = "History of COVID-19 infection"
groups_dict = {
    "No": {"label": "No Exposure (HDs)", "color": "#00D5E5"},
    "Yes": {"label": "Previous COVID-19 (HDs)", "color": "#FF1F5b"},
}

def perform_chi_square(data, col1, col2):
    contingency_table = pd.crosstab(data[col1], data[col2])
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    return p_value

fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(20, 10), sharey=False)
plt.subplots_adjust(hspace=0.3, wspace=0.3)

bar_width = 0.35  # Width of the bars

for i, columns in enumerate(column_sets):
    counts_dict = {}
    for group_value, group_info in groups_dict.items():
        group_data = data_HD[data_HD[stratification_column] == group_value]
        counts = group_data[columns].apply(pd.Series.value_counts).T.fillna(0)
        counts.index.name = 'Timepoint'
        counts.reset_index(inplace=True)
        counts['positive%'] = counts['positive'] / (counts['positive'] + counts['negative']) * 100
        counts['negative%'] = counts['negative'] / (counts['positive'] + counts['negative']) * 100
        counts_dict[group_value] = counts

    x_axs, y_axs = divmod(i, 2)
    for group_index, (group_value, counts) in enumerate(counts_dict.items()):
        countspercent = counts.drop(columns=['positive', 'negative'])
        countspercent.plot(kind='bar', stacked=True,
                           ax=axs[x_axs, y_axs],
                           color=[groups_dict[group_value]['color'], '#CDCDCD'],
                           position=group_index-0.5,
                           width=bar_width,
                           label=groups_dict[group_value]['label'])

    axs[x_axs, y_axs].set_title(f'{name_set[i]}', fontsize=20)
    axs[x_axs, y_axs].set_xlabel(' ', fontsize=15)
    axs[x_axs, y_axs].set_ylabel('Percentage', fontsize=15)
    axs[x_axs, y_axs].tick_params(axis='both', labelsize=15)
    axs[x_axs, y_axs].set_xticklabels(['Pre', '2d Post D1', '21d Post D1', '2d Post D2', '21d Post D2', '60d Post D2'], rotation=25, ha='right')

    for group_index, (group_value, counts) in enumerate(counts_dict.items()):
        for container in axs[x_axs, y_axs].containers:
            axs[x_axs, y_axs].bar_label(container, labels=[f"{int(v)}" for v in container.datavalues], label_type='center', fontsize=10, color='black')

    # Perform chi-square test between adjacent time points
    for k in range(len(columns) - 1):
        p_value = perform_chi_square(data_HD, columns[k], columns[k+1])
        significance = ''
        if p_value < 0.001:
            significance = '***'
        elif p_value < 0.01:
            significance = '**'
        elif p_value < 0.05:
            significance = '*'
        elif p_value >= 0.05:
            significance = 'ns'
        
        if significance:
            x1 = k - bar_width/2
            x2 = k + 1 - bar_width/2
            y = axs[x_axs, y_axs].get_ylim()[1] * 1.05
            axs[x_axs, y_axs].annotate(significance, 
                                       xy=((x1+x2)/2, y),
                                       xytext=(0, 0),
                                       textcoords="offset points",
                                       ha='center',
                                       va='center',
                                       fontsize=20,
                                       color='red')

plt.tight_layout()
plt.show()

# Save the figure if needed
save_figure(plot=fig, destination_path=figure_destination_path, figure_name="Figure 4b")


## [OLD] Supp Fig S3a: HD vs Healthy in all

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Define new column sets and parameters for healthy vs control comparison
column_sets = [
    ['IgGpost21D1', 'IgGpost21D2'],
    ['Neutrolizing21D1', 'Neutrolizing21D2'],
    ['AntiSpike21D1', 'AntiSpike21D2'],
    ['AntiRBD21D1', 'AntiRBD21D2']
]
name_set = ['IgG', 'Neutralizing Ab', 'Anti Spike Ab', 'Anti RBD']
unit_set = ['Index', 'micg/ml', 'RU/mL', 'Index']
cutoff_set = [1.1, 2.5, 8, 1.1]

fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(40, 25), sharey=False)
x_axs = 0
y_axs = 0

for i, columns in enumerate(column_sets):
    melted_data = pd.melt(data, id_vars='case/control', value_vars=columns, var_name='Timepoint', value_name='Value')
    sns.violinplot(x='Timepoint', y='Value', hue="case/control", data=melted_data,
                   ax=axs[x_axs, y_axs],
                   split=False,
                   palette=["#3C93C2", "#40AD5A"],
                   linewidth=5)
    axs[x_axs, y_axs].set_title(f'{name_set[i]}', fontsize=55)
    axs[x_axs, y_axs].set_xlabel(' ', fontsize=35)
    axs[x_axs, y_axs].set_ylabel(f'{unit_set[i]}', fontsize=35)
    axs[x_axs, y_axs].tick_params(axis='both', labelsize=30)
    axs[x_axs, y_axs].axhline(y=cutoff_set[i], linestyle=':', color='white', linewidth=7)
    axs[x_axs, y_axs].set_ylim(0)

    x_axs += 1
    if x_axs > 1:
        x_axs = 0
        y_axs += 1

plt.tight_layout()
plt.show()


In [ ]:
df = data.copy()
# Extract relevant columns with the correct names
relevant_columns_corrected = ['case/control', 'AntiSpike21D1POS', 'AntiSpike21D2POS']
data_corrected = df[relevant_columns_corrected]

# Count positive and negative strings for AntiSpike21D1POS and AntiSpike21D2POS in HD case and healthy control
counts_corrected = data_corrected.groupby(['case/control']).apply(
    lambda x: pd.Series({
        'AntiSpike21D1POS_positive': (x['AntiSpike21D1POS'] == 'positive').sum(),
        'AntiSpike21D1POS_negative': (x['AntiSpike21D1POS'] == 'negative').sum(),
        'AntiSpike21D2POS_positive': (x['AntiSpike21D2POS'] == 'positive').sum(),
        'AntiSpike21D2POS_negative': (x['AntiSpike21D2POS'] == 'negative').sum()
    })
).reset_index()

# Perform chi-square test
observed_21D1_corrected = counts_corrected[['AntiSpike21D1POS_positive', 'AntiSpike21D1POS_negative']].values
observed_21D2_corrected = counts_corrected[['AntiSpike21D2POS_positive', 'AntiSpike21D2POS_negative']].values

chi2_21D1_corrected, p_21D1_corrected, _, _ = chi2_contingency(observed_21D1_corrected)
chi2_21D2_corrected, p_21D2_corrected, _, _ = chi2_contingency(observed_21D2_corrected)

# Plot the results
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Plot for AntiSpike21D1POS
ax[0].bar(['HD case', 'Healthy control'], counts_corrected['AntiSpike21D1POS_positive'], label='Positive')
ax[0].bar(['HD case', 'Healthy control'], counts_corrected['AntiSpike21D1POS_negative'], bottom=counts_corrected['AntiSpike21D1POS_positive'], label='Negative')
ax[0].set_title(f'AntiSpike21D1POS\nChi-square: {chi2_21D1_corrected:.2f}, p-value: {p_21D1_corrected:.2e}')
ax[0].set_ylabel('Counts')
ax[0].legend()

# Plot for AntiSpike21D2POS
ax[1].bar(['HD case', 'Healthy control'], counts_corrected['AntiSpike21D2POS_positive'], label='Positive')
ax[1].bar(['HD case', 'Healthy control'], counts_corrected['AntiSpike21D2POS_negative'], bottom=counts_corrected['AntiSpike21D2POS_positive'], label='Negative')
ax[1].set_title(f'AntiSpike21D2POS\nChi-square: {chi2_21D2_corrected:.2f}, p-value: {p_21D2_corrected:.2e}')
ax[1].set_ylabel('Counts')
ax[1].legend()

plt.tight_layout()
plt.show()



In [ ]:
# Plot the results with p-value and chi-square statistic on top of each bar plot
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Plot for AntiSpike21D1POS
ax[0].bar(['HD case', 'Healthy control'], counts_corrected['AntiSpike21D1POS_positive'], label='Positive')
ax[0].bar(['HD case', 'Healthy control'], counts_corrected['AntiSpike21D1POS_negative'], bottom=counts_corrected['AntiSpike21D1POS_positive'], label='Negative')
ax[0].set_title('AntiSpike21D1POS')
ax[0].set_ylabel('Counts')
ax[0].legend()
ax[0].text(0.5, counts_corrected['AntiSpike21D1POS_positive'].max() + 5, f'Chi-square: {chi2_21D1_corrected:.2f}\np-value: {p_21D1_corrected:.2e}', ha='center')

# Plot for AntiSpike21D2POS
ax[1].bar(['HD case', 'Healthy control'], counts_corrected['AntiSpike21D2POS_positive'], label='Positive')
ax[1].bar(['HD case', 'Healthy control'], counts_corrected['AntiSpike21D2POS_negative'], bottom=counts_corrected['AntiSpike21D2POS_positive'], label='Negative')
ax[1].set_title('AntiSpike21D2POS')
ax[1].set_ylabel('Counts')
ax[1].legend()
ax[1].text(0.5, counts_corrected['AntiSpike21D2POS_positive'].max() + 5, f'Chi-square: {chi2_21D2_corrected:.2f}\np-value: {p_21D2_corrected:.2e}', ha='center')

plt.tight_layout()
plt.show()


## [NEW] Fig 5a. HD vs Healthy no exposure

In [ ]:
# Change the sns bar type to violonplot in plot_comparison_boxplots_auto

In [ ]:
# All HD {no previous COVID vs previous COVID}
data = pd.read_csv(file_path)
data_NoCOVID19 = data[data["History of COVID-19 infection"] == "No"]


column_sets = [
    ['IgGpost21D1', 'IgGpost21D2'],
    ['Neutrolizing21D1', 'Neutrolizing21D2'],
    ['AntiSpike21D1', 'AntiSpike21D2'],
    ['AntiRBD21D1', 'AntiRBD21D2']
]
name_set = ['IgG', 'Neutralizing Ab', 'Anti Spike Ab', 'Anti RBD']
unit_set = ['Index', 'micg/ml', 'RU/mL', 'Index']
cutoff_set = [1.1, 2.5, 8, 1.1]
xticks = ['Pre', '1or2D1', '21D1', '1or2D2', '21D2', '60D2']

stratification_column=r"case/control"
groups_dict = {
    "HD case": {"label in figure": "HD", "color of boxplot": "#00D5E5"},
    "Healthy control": {"label in figure": "Healthy", "color of boxplot": "#9ACD32"},
}

statistical_test= "mannwhitney" #"mannwhitney" OR "ttest"
fig_size=(40,40)
fig = plot_comparison_boxplots_auto(data_NoCOVID19, column_sets, name_set, unit_set, cutoff_set, xticks, 
                               stratification_column=stratification_column, groups_dict=groups_dict, statistical_test=statistical_test, fig_size=fig_size)
save_figure(plot=fig, destination_path=figure_destination_path, figure_name="Figure 5a")

## [NEW] Fig 5b. HD vs Healthy no exposure

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency

# Load data
data = pd.read_csv(file_path)
data_noCOVID19 = data[data["History of COVID-19 infection"] == "No"]

column_sets = [
    ['IgG positive 21D1', 'IgG positive 21D2'],
    ['Neutrolizing21D1Pos', 'Neutrolizing21D2Pos'],    
    ['AntiSpike21D1POS', 'AntiSpike21D2POS'],
    [ 'AntiRBD21D1POS', 'AntiRBD21D2POS', ]
]
name_set = ['IgG', 'Neutralizing Ab', 'Anti Spike Ab',  'Anti RBD']
stratification_column = r"case/control"
groups_dict = {
    "Healthy control": {"label": "Healthy", "color": "#9ACD32"},
    "HD case": {"label": "HD", "color": "#00D5E5"},
}


def perform_chi_square(data, col1, col2):
    contingency_table = pd.crosstab(data[col1], data[col2])
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    return p_value

fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(20, 10), sharey=False)
plt.subplots_adjust(hspace=0.3, wspace=0.3)

bar_width = 0.35  # Width of the bars

for i, columns in enumerate(column_sets):
    counts_dict = {}
    for group_value, group_info in groups_dict.items():
        group_data = data_noCOVID19[data_noCOVID19[stratification_column] == group_value]
        counts = group_data[columns].apply(pd.Series.value_counts).T.fillna(0)
        counts.index.name = 'Timepoint'
        counts.reset_index(inplace=True)
        counts['positive%'] = counts['positive'] / (counts['positive'] + counts['negative']) * 100
        counts['negative%'] = counts['negative'] / (counts['positive'] + counts['negative']) * 100
        counts_dict[group_value] = counts

    x_axs, y_axs = divmod(i, 2)
    for group_index, (group_value, counts) in enumerate(counts_dict.items()):
        countspercent = counts.drop(columns=['positive', 'negative'])
        countspercent.plot(kind='bar', stacked=True,
                           ax=axs[x_axs, y_axs],
                           color=[groups_dict[group_value]['color'], '#CDCDCD'],
                           position=group_index-0.5,
                           width=bar_width,
                           label=groups_dict[group_value]['label'])

    axs[x_axs, y_axs].set_title(f'{name_set[i]}', fontsize=20)
    axs[x_axs, y_axs].set_xlabel(' ', fontsize=15)
    axs[x_axs, y_axs].set_ylabel('Percentage', fontsize=15)
    axs[x_axs, y_axs].tick_params(axis='both', labelsize=15)
    axs[x_axs, y_axs].set_xticklabels(['21d Post D1', '21d Post D2'], rotation=25, ha='right')

    for group_index, (group_value, counts) in enumerate(counts_dict.items()):
        for container in axs[x_axs, y_axs].containers:
            axs[x_axs, y_axs].bar_label(container, labels=[f"{int(v)}" for v in container.datavalues], label_type='center', fontsize=10, color='black')

    # Perform chi-square test between adjacent time points
    for k in range(len(columns) - 1):
        p_value = perform_chi_square(data_noCOVID19, columns[k], columns[k+1])
        significance = ''
        if p_value < 0.001:
            significance = '***'
        elif p_value < 0.01:
            significance = '**'
        elif p_value < 0.05:
            significance = '*'
        elif p_value >= 0.05:
            significance = 'ns'
        
        if significance:
            x1 = k - bar_width/2
            x2 = k + 1 - bar_width/2
            y = axs[x_axs, y_axs].get_ylim()[1] * 1.05
            axs[x_axs, y_axs].annotate(significance, 
                                       xy=((x1+x2)/2, y),
                                       xytext=(0, 0),
                                       textcoords="offset points",
                                       ha='center',
                                       va='center',
                                       fontsize=20,
                                       color='red')

plt.tight_layout()
plt.show()

# Save the figure if needed
save_figure(plot=fig, destination_path=figure_destination_path, figure_name="Figure 5b")


## [NEW] Supp Fig S3a: HD vs Healthy with COVID-19 Infection

In [ ]:
# All HD {no previous COVID vs previous COVID}
data = pd.read_csv(file_path)
data_NoCOVID19 = data[data["History of COVID-19 infection"] == "Yes"]


column_sets = [
    ['IgGpost21D1', 'IgGpost21D2'],
    ['Neutrolizing21D1', 'Neutrolizing21D2'],
    ['AntiSpike21D1', 'AntiSpike21D2'],
    ['AntiRBD21D1', 'AntiRBD21D2']
]
name_set = ['IgG', 'Neutralizing Ab', 'Anti Spike Ab', 'Anti RBD']
unit_set = ['Index', 'micg/ml', 'RU/mL', 'Index']
cutoff_set = [1.1, 2.5, 8, 1.1]
xticks = ['Pre', '1or2D1', '21D1', '1or2D2', '21D2', '60D2']

stratification_column=r"case/control"
groups_dict = {
    "HD case": {"label in figure": "HD", "color of boxplot": "#00D5E5"},
    "Healthy control": {"label in figure": "Healthy", "color of boxplot": "#9ACD32"},
}

statistical_test= "mannwhitney" #"mannwhitney" OR "ttest"
fig_size=(40,40)
fig = plot_comparison_boxplots_auto(data_NoCOVID19, column_sets, name_set, unit_set, cutoff_set, xticks, 
                               stratification_column=stratification_column, groups_dict=groups_dict, statistical_test=statistical_test, fig_size=fig_size)
save_figure(plot=fig, destination_path=figure_destination_path, figure_name="Supplemantary Figure S3a")

## [NEW] Supp Fig S3a: HD vs Healthy with COVID-19 Infection

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency

# Load data
data = pd.read_csv(file_path)
data_noCOVID19 = data[data["History of COVID-19 infection"] == "Yes"]

column_sets = [
    ['IgG positive 21D1', 'IgG positive 21D2'],
    ['Neutrolizing21D1Pos', 'Neutrolizing21D2Pos'],    
    ['AntiSpike21D1POS', 'AntiSpike21D2POS'],
    [ 'AntiRBD21D1POS', 'AntiRBD21D2POS', ]
]
name_set = ['IgG', 'Neutralizing Ab', 'Anti Spike Ab',  'Anti RBD']
stratification_column = r"case/control"
groups_dict = {
    "Healthy control": {"label": "Healthy", "color": "#9ACD32"},
    "HD case": {"label": "HD", "color": "#00D5E5"},
}


def perform_chi_square(data, col1, col2):
    contingency_table = pd.crosstab(data[col1], data[col2])
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    return p_value

fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(20, 10), sharey=False)
plt.subplots_adjust(hspace=0.3, wspace=0.3)

bar_width = 0.35  # Width of the bars

for i, columns in enumerate(column_sets):
    counts_dict = {}
    for group_value, group_info in groups_dict.items():
        group_data = data_noCOVID19[data_noCOVID19[stratification_column] == group_value]
        counts = group_data[columns].apply(pd.Series.value_counts).T.fillna(0)
        counts.index.name = 'Timepoint'
        counts.reset_index(inplace=True)
        counts['positive%'] = counts['positive'] / (counts['positive'] + counts['negative']) * 100
        counts['negative%'] = counts['negative'] / (counts['positive'] + counts['negative']) * 100
        counts_dict[group_value] = counts

    x_axs, y_axs = divmod(i, 2)
    for group_index, (group_value, counts) in enumerate(counts_dict.items()):
        countspercent = counts.drop(columns=['positive', 'negative'])
        countspercent.plot(kind='bar', stacked=True,
                           ax=axs[x_axs, y_axs],
                           color=[groups_dict[group_value]['color'], '#CDCDCD'],
                           position=group_index-0.5,
                           width=bar_width,
                           label=groups_dict[group_value]['label'])

    axs[x_axs, y_axs].set_title(f'{name_set[i]}', fontsize=20)
    axs[x_axs, y_axs].set_xlabel(' ', fontsize=15)
    axs[x_axs, y_axs].set_ylabel('Percentage', fontsize=15)
    axs[x_axs, y_axs].tick_params(axis='both', labelsize=15)
    axs[x_axs, y_axs].set_xticklabels(['21d Post D1', '21d Post D2'], rotation=25, ha='right')

    for group_index, (group_value, counts) in enumerate(counts_dict.items()):
        for container in axs[x_axs, y_axs].containers:
            axs[x_axs, y_axs].bar_label(container, labels=[f"{int(v)}" for v in container.datavalues], label_type='center', fontsize=10, color='black')

    # Perform chi-square test between adjacent time points
    for k in range(len(columns) - 1):
        p_value = perform_chi_square(data_noCOVID19, columns[k], columns[k+1])
        significance = ''
        if p_value < 0.001:
            significance = '***'
        elif p_value < 0.01:
            significance = '**'
        elif p_value < 0.05:
            significance = '*'
        elif p_value >= 0.05:
            significance = 'ns'
        
        if significance:
            x1 = k - bar_width/2
            x2 = k + 1 - bar_width/2
            y = axs[x_axs, y_axs].get_ylim()[1] * 1.05
            axs[x_axs, y_axs].annotate(significance, 
                                       xy=((x1+x2)/2, y),
                                       xytext=(0, 0),
                                       textcoords="offset points",
                                       ha='center',
                                       va='center',
                                       fontsize=20,
                                       color='red')

plt.tight_layout()
plt.show()

# Save the figure if needed
save_figure(plot=fig, destination_path=figure_destination_path, figure_name="Supplemantary Figure S3b")


# Cellular

## IGRA: BTI vs no BTI

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_excel(r"C:\Users\LEGION\OneDrive\Maghale\Vaccination in dialysi\Submit\E12 Final Vaghei.xlsx")

# Separate the data into two groups and drop NaN values
with_dose3 = df[df["D3-variable 2-Type"].notna()]["IGRA_14Month_PostD2"].dropna()
without_dose3 = df[df["D3-variable 2-Type"].isna()]["IGRA_14Month_PostD2"].dropna()

# Perform a statistical test (Mann-Whitney U test, suitable for non-normally distributed data)
stat, p_value = stats.mannwhitneyu(with_dose3, without_dose3)

# Print the statistical test results
print(f'Mann-Whitney U Test: U statistic={stat:.2f}, p-value={p_value:.4f}')

# Create the boxplot
plt.figure(figsize=(5, 4))
sns.boxplot(x=df["D3-variable 2-Type"].notna(), y=df["IGRA_14Month_PostD2"])
sns.stripplot(x=df["D3-variable 2-Type"].notna(), y=df["IGRA_14Month_PostD2"], color=".3", jitter=True)

# Annotate the plot with the p-value
plt.title(f'Comparison of IGRA Values by Third Dose Status\nMann-Whitney U Test p-value = {p_value:.4f}')
plt.xlabel('Received Third Dose')
plt.ylabel('IGRA 14 Month PostD2')
plt.xticks([0, 1], ['No', 'Yes'])
plt.grid(True)

# Show the plot
plt.tight_layout()
plt.show()



## IGRA and Heumoral Correlation

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming the dataset is in a CSV file named 'data.csv'
# df = pd.read_csv('data.csv')

df = pd.read_excel(r"C:\Users\LEGION\OneDrive\Maghale\Vaccination in dialysi\Submit\E12 Final Vaghei.xlsx")
df = df[df["History of COVID-19 infection"] == "No"]
# Define the pairs
pairs = [
    ("IgGpost21D2", "IgGpost60D2"),
    ("AntiSpike21D2", "AntiSpike60D2"),
    ("Neutrolizing21D2", "Neutrolizing60D2"),
    ("AntiRBD21D2", "AntiRBD60D2")
]

# Prepare the combined values for each pair
for pair in pairs:
    df[pair[0] + "_final"] = df[pair[0]].combine_first(df[pair[1]])

# Define the subplots
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

# Analyze and plot each pair
for idx, pair in enumerate(pairs):
    humoral_test = df[pair[0] + "_final"]
    igra_test = df["IGRA_14Month_PostD2"]

    # Remove NaN values
    mask = ~humoral_test.isna() & ~igra_test.isna()
    humoral_test = humoral_test[mask]
    igra_test = igra_test[mask]
        
    # Calculate correlation
    correlation, p_value = stats.spearmanr(igra_test, humoral_test)
    
    # Create the plot
    sns.regplot(x=humoral_test, y=igra_test, ax=axes[idx], scatter_kws={'s':20}, line_kws={'color':'red'})
    axes[idx].set_title(f'{pair[0].split("21D2")[0]} Correlation\nSpearman rho = {correlation:.2f}, p-value = {p_value:.4f}')
    axes[idx].set_xlabel(f'{pair[0].split("21D2")[0]} Test Value')
    axes[idx].set_ylabel('IGRA 14 Month PostD2')
    
plt.tight_layout()
plt.show()




In [ ]:
# Prepare the combined values for each pair
for pair in pairs:
    df[pair[0] + "_final"] = df[pair[0]].combine_first(df[pair[1]])

# Separate the patients into two groups
df_with_dose3 = df[df["D3-variable 2-Type"].notna()]
df_without_dose3 = df[df["D3-variable 2-Type"].isna()]

# Function to create the 2x2 plot
def create_plot(df, title):
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    axes = axes.flatten()

    for idx, pair in enumerate(pairs):
        humoral_test = df[pair[0] + "_final"]
        igra_test = df["IGRA_14Month_PostD2"]
        
        # Remove NaN values
        mask = ~humoral_test.isna() & ~igra_test.isna()
        humoral_test = humoral_test[mask]
        igra_test = igra_test[mask]
        
        # Calculate correlation
        correlation, p_value = stats.spearmanr(igra_test, humoral_test)
        
        # Create the plot
        sns.regplot(x=humoral_test, y=igra_test, ax=axes[idx], scatter_kws={'s':20}, line_kws={'color':'red'})
        axes[idx].set_title(f'{pair[0].split("21D2")[0]} Correlation\nSpearman rho = {correlation:.2f}, p-value = {p_value:.4f}')
        axes[idx].set_xlabel(f'{pair[0].split("21D2")[0]} Test Value')
        axes[idx].set_ylabel('IGRA 14 Month PostD2')
        
        save_figure(plot=fig, destination_path=figure_destination_path, figure_name=f"{title}")

    plt.suptitle(title)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()
    plt.savefig

# Create the plots for both groups
create_plot(df_with_dose3, '6a')
create_plot(df_without_dose3, '6b')

# Regression

## FUNCTION

In [ ]:
# for HD without COVID-19 

data = pd.read_csv(file_path)
data_HD_NoPreviousCOVID =  data[(data["case/control"] == "HD case") & (data["History of COVID-19 infection"] == "No")]
#data_HD_gold = data[(data["case/control"] == "HD case") & (data["HD Cohort Type"] == "with HD0")]

column_sets = [
    ['IgGpost21D2', 'IgGpost60D2'],
    ['Neutrolizing21D2', 'Neutrolizing60D2'],
    ['AntiSpike21D2', 'AntiSpike60D2'],
    ['AntiRBD21D2', 'AntiRBD60D2']
]


# calculate 21 and 60 average
def create_average_column(df, col1, col2, new_col_name='average_col'):
    df[new_col_name] = df.apply(
        lambda row: np.nan if pd.isna(row[col1]) and pd.isna(row[col2])
        else row[col1] if pd.isna(row[col2])
        else row[col2] if pd.isna(row[col1])
        else (row[col1] + row[col2]) / 2, axis=1
    )
    return df

data_HD_NoPreviousCOVID_average=data_HD_NoPreviousCOVID
average_col_names=[]
for col1, col2 in column_sets:
    new_col_name=col1[:-4]
    if new_col_name=='IgGpost':
        new_col_name='IgG'
    average_col_names.append(new_col_name)
    data_HD_NoPreviousCOVID_average = create_average_column(data_HD_NoPreviousCOVID_average, col1, col2, new_col_name)
data_HD_NoPreviousCOVID_average


In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import summary_table

def regression_analysis_with_imputation(df, target_col, input_cols, KNN_imputation=False):
    if KNN_imputation:
        best_n = None
        best_score = -np.inf
        for n in [3, 5, 7, 9, 11]:
            imputer = KNNImputer(n_neighbors=n)
            df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
            X = df_imputed[input_cols]
            y = df_imputed[target_col]
            model = LinearRegression()
            model.fit(X, y)
            score = model.score(X, y)
            if score > best_score:
                best_score = score
                best_n = n
        print(f"Best K for KNN imputation: {best_n}")
        imputer = KNNImputer(n_neighbors=best_n)
        df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
    
    results = []
    
    for col in input_cols:
        non_missing_df = df[[target_col, col]].dropna()
        X = sm.add_constant(non_missing_df[col])
        y = non_missing_df[target_col]
        model = sm.OLS(y, X).fit()
        st, data, ss2 = summary_table(model, alpha=0.05)
        beta = model.params[col]
        conf_int = model.conf_int().loc[col].tolist()
        p_value = model.pvalues[col]
        results.append({
            'Input Column': col,
            'Beta': beta,
            '95% CI Lower': conf_int[0],
            '95% CI Upper': conf_int[1],
            'p-value': p_value
        })
    
    results_df = pd.DataFrame(results)
    return results_df

# Example usage:
i =0

result_table = regression_analysis_with_imputation(data_HD_NoPreviousCOVID_average, average_col_names[i], ['input_column1', 'input_column2'], KNN_imputation=True)
#result_table


In [ ]:
average_col_names

# DATA for negative ANTISPIKE21D1POS

In [ ]:
# Exclude cases where AntiSpike21D1POS is positive
filtered_data = data_corrected[data_corrected['AntiSpike21D1POS'] != 'positive']

# Count positive and negative strings for AntiSpike21D2POS in HD case and healthy control after filtering
filtered_counts = filtered_data.groupby(['case/control']).apply(
    lambda x: pd.Series({
        'AntiSpike21D2POS_positive': (x['AntiSpike21D2POS'] == 'positive').sum(),
        'AntiSpike21D2POS_negative': (x['AntiSpike21D2POS'] == 'negative').sum()
    })
).reset_index()

# Perform chi-square test on filtered data for AntiSpike21D2POS
observed_21D2_filtered = filtered_counts[['AntiSpike21D2POS_positive', 'AntiSpike21D2POS_negative']].values

chi2_21D2_filtered, p_21D2_filtered, _, _ = chi2_contingency(observed_21D2_filtered)

# Plot the results for AntiSpike21D2POS after filtering
plt.figure(figsize=(6, 6))

plt.bar(['HD case', 'Healthy control'], filtered_counts['AntiSpike21D2POS_positive'], label='Positive')
plt.bar(['HD case', 'Healthy control'], filtered_counts['AntiSpike21D2POS_negative'], bottom=filtered_counts['AntiSpike21D2POS_positive'], label='Negative')
plt.title(f'AntiSpike21D2POS (Filtered)\nChi-square: {chi2_21D2_filtered:.2f}, p-value: {p_21D2_filtered:.2e}')
plt.ylabel('Counts')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Filter data for cases with negative AntiSpike21D1POS
negative_data = data[data['AntiSpike21D1POS'] == 'negative']

# Define new column sets and parameters for healthy vs control comparison
column_sets = [
    ['IgGpost21D1', 'IgGpost21D2'],
    ['Neutrolizing21D1', 'Neutrolizing21D2'],
    ['AntiSpike21D1', 'AntiSpike21D2'],
    ['AntiRBD21D1', 'AntiRBD21D2']
]
name_set = ['IgG', 'Neutralizing Ab', 'Anti Spike Ab', 'Anti RBD']
unit_set = ['Index', 'micg/ml', 'RU/mL', 'Index']
cutoff_set = [1.1, 2.5, 8, 1.1]

fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(24, 16), sharey=False)
x_axs = 0
y_axs = 0

for i, columns in enumerate(column_sets):
    melted_data = pd.melt(negative_data, id_vars='case/control', value_vars=columns, var_name='Timepoint', value_name='Value')
    sns.barplot(x='Timepoint', y='Value', hue='case/control', data=melted_data, ax=axs[x_axs, y_axs],
                palette=["#3C93C2", "#40AD5A"], ci='sd', capsize=0.1, errwidth=2)
    
    axs[x_axs, y_axs].set_title(f'{name_set[i]}', fontsize=24, weight='bold')
    axs[x_axs, y_axs].set_xlabel('Timepoint', fontsize=20)
    axs[x_axs, y_axs].set_ylabel(f'{unit_set[i]}', fontsize=20)
    axs[x_axs, y_axs].tick_params(axis='both', labelsize=18)
    axs[x_axs, y_axs].axhline(y=cutoff_set[i], linestyle='--', color='gray', linewidth=2)
    axs[x_axs, y_axs].legend(title='Case/Control', title_fontsize='13', fontsize='12')
    axs[x_axs, y_axs].set_ylim(0)

    x_axs += 1
    if x_axs > 1:
        x_axs = 0
        y_axs += 1

plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import ttest_ind

# Sample data (You need to replace this with your actual data)
# data = pd.read_csv('your_data_file.csv')

# Filter data for cases with negative AntiSpike21D1POS
negative_data = data[data['AntiSpike21D1POS'] == 'negative']

# Define new column sets and parameters for healthy vs control comparison
column_sets = [
    ['IgGpost21D1', 'IgGpost21D2'],
    ['Neutrolizing21D1', 'Neutrolizing21D2'],
    ['AntiSpike21D1', 'AntiSpike21D2'],
    ['AntiRBD21D1', 'AntiRBD21D2']
]
name_set = ['IgG', 'Neutralizing Ab', 'Anti Spike Ab', 'Anti RBD']
unit_set = ['Index', 'micg/ml', 'RU/mL', 'Index']
cutoff_set = [1.1, 2.5, 8, 1.1]

fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(24, 16), sharey=False)
x_axs = 0
y_axs = 0

for i, columns in enumerate(column_sets):
    melted_data = pd.melt(negative_data, id_vars='case/control', value_vars=columns, var_name='Timepoint', value_name='Value')
    sns.barplot(x='Timepoint', y='Value', hue='case/control', data=melted_data, ax=axs[x_axs, y_axs],
                palette=["#3C93C2", "#40AD5A"], ci='sd', capsize=0.1, errwidth=2)
    
    axs[x_axs, y_axs].set_title(f'{name_set[i]}', fontsize=24, weight='bold')
    axs[x_axs, y_axs].set_xlabel('Timepoint', fontsize=20)
    axs[x_axs, y_axs].set_ylabel(f'{unit_set[i]}', fontsize=20)
    axs[x_axs, y_axs].tick_params(axis='both', labelsize=18)
    axs[x_axs, y_axs].axhline(y=cutoff_set[i], linestyle='--', color='gray', linewidth=2)
    axs[x_axs, y_axs].legend(title='Case/Control', title_fontsize='13', fontsize='12')
    axs[x_axs, y_axs].set_ylim(0)
    
    # Perform t-tests, calculate mean differences, and annotate with p-values and mean differences
    timepoints = melted_data['Timepoint'].unique()
    for timepoint in timepoints:
        case_values = melted_data[(melted_data['Timepoint'] == timepoint) & (melted_data['case/control'] == 'case')]['Value']
        control_values = melted_data[(melted_data['Timepoint'] == timepoint) & (melted_data['case/control'] == 'control')]['Value']
        
        if not case_values.empty and not control_values.empty:
            t_stat, p_val = ttest_ind(case_values.dropna(), control_values.dropna(), nan_policy='omit')
            mean_diff = case_values.mean() - control_values.mean()
            # Annotate p-values and mean differences
            x_pos = timepoints.tolist().index(timepoint)
            y_max = melted_data[melted_data['Timepoint'] == timepoint]['Value'].max()
            y_pos = y_max + 0.1 * y_max
            axs[x_axs, y_axs].text(x_pos, y_pos, f'p={p_val:.3e}\nΔ={mean_diff:.2f}', ha='center', fontsize=14, color='black')
        else:
            x_pos = timepoints.tolist().index(timepoint)
            y_max = melted_data[melted_data['Timepoint'] == timepoint]['Value'].max()
            y_pos = y_max + 0.1 * y_max
            axs[x_axs, y_axs].text(x_pos, y_pos, 'p=N/A\nΔ=N/A', ha='center', fontsize=14, color='black')

    x_axs += 1
    if x_axs > 1:
        x_axs = 0
        y_axs += 1

plt.tight_layout()
plt.show()


# My questions

### Previous COVID and Anti-spike

In [ ]:
#--------------variables for code-------------
figure_name="Immunogenicity_HD_Gold_allIncludingSpikePositive"
data_HD= data[(data["case/control"]=="HD case")]
data_HD_previousCOVID = data_HD[data_HD["History of COVID-19 infection"] == "YES"]


# Define column sets and other parameters
data_4figure = data_HD_gold

column_sets = [
    ['IgGPre', 'IgGpost1or2D1', 'IgGpost21D1', 'IgGpost1or2D2', 'IgGpost21D2', 'IgGpost60D2'],
    ['NeutrolizingPre', 'Neutrolizing1or2D1', 'Neutrolizing21D1', 'Neutrolizing1or2D2', 'Neutrolizing21D2', 'Neutrolizing60D2'],
    ['AntiSpikepre', 'AntiSpike1or2D1', 'AntiSpike21D1', 'AntiSpike1or2D2', 'AntiSpike21D2', 'AntiSpike60D2'],
    ['AntiRBDpre', 'AntiRBD1or2D1', 'AntiRBD21D1', 'AntiRBD1or2D2', 'AntiRBD21D2', 'AntiRBD60D2']
]
name_set = ['IgG', 'Neutralizing Ab', 'Anti Spike Ab', 'Anti RBD']
unit_set = ['Index', 'micg/ml', 'RU/mL', 'Index']
cutoff_set = [1.1, 2.5, 8, 1.1]
xticks = ['Pre', '1or2D1', '21D1', '1or2D2', '21D2', '60D2']
color_palete=["#00D5E5", "#00D5E5", "#3C93C2", "#00D5E5", "#3C93C2", "#3C93C2"]

#--------------code-------------
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel, wilcoxon, sem
import itertools




# Define a function to calculate 95% confidence interval
def calc_95ci(data1, data2):
    diff = data1 - data2
    mean_diff = np.mean(diff)
    ci = 1.96 * sem(diff)
    return mean_diff - ci, mean_diff + ci


fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(40, 40), sharey=False)
x_axs = 0
y_axs = 0

for i, columns in enumerate(column_sets):
    melted_data = pd.melt(data_4figure, value_vars=columns, var_name='Timepoint', value_name='Value')

    sns.boxplot(x='Timepoint', y='Value', data=melted_data,
                ax=axs[x_axs, y_axs],
                palette=["#00D5E5", "#00D5E5", "#00D5E5", "#00D5E5", "#00D5E5", "#00D5E5"],
                linewidth=5)


    axs[x_axs, y_axs].axhline(y=cutoff_set[i], linestyle=':', color='grey', linewidth=7)

    sns.stripplot(x='Timepoint', y='Value', data=melted_data,
                  ax=axs[x_axs, y_axs],
                  palette=["black"],
                  size=13,
                  alpha=0.5)
    axs[x_axs, y_axs].set_title(f'{name_set[i]}', fontsize=55)
    axs[x_axs, y_axs].set_xlabel(' ', fontsize=35)
    axs[x_axs, y_axs].set_ylabel(f'{unit_set[i]}', fontsize=35)
    axs[x_axs, y_axs].tick_params(axis='both', labelsize=30)
    axs[x_axs, y_axs].set_xticklabels(xticks)

    bottom, top = axs[x_axs, y_axs].get_ylim()
    y_range = top - bottom

    combinations = itertools.combinations(columns, 2)
    significant_combinations = []
    for combination in combinations:
        # Extract data for the two time points
        valid_comb_data = data.dropna(subset=[combination[0], combination[1]])
        tp1 = valid_comb_data[combination[0]]
        tp2 = valid_comb_data[combination[1]]
        t_stat, p_value = wilcoxon(tp1, tp2)
        ci_low, ci_high = calc_95ci(tp1, tp2)
        significant_combinations.append([combination, p_value, (ci_low, ci_high)])

    for j, significant_combination in enumerate(significant_combinations):
        x1 = columns.index(significant_combination[0][0])
        x2 = columns.index(significant_combination[0][1])
        level = len(significant_combinations) - j
        bar_height = (y_range * 0.1 * level) + top
        bar_tips = bar_height - (y_range * 0.02)
        axs[x_axs, y_axs].plot(
            [x1, x1, x2, x2],
            [bar_tips, bar_height, bar_height, bar_tips], lw=1, c='k'
        )

        p = significant_combination[1]
        if p < 0.001:
            sig_symbol = '***'
        elif p < 0.01:
            sig_symbol = '**'
        elif p < 0.05:
            sig_symbol = '*'
        else:
            sig_symbol = 'NS'

        CI95 = significant_combination[2]
        text4plot = f"[{CI95[0]:.2f}, {CI95[1]:.2f}] {sig_symbol}"
        text_height = bar_height + (y_range * 0.01)
        axs[x_axs, y_axs].text((x1 + x2) / 2, text_height, text4plot, ha='center', va='bottom', c='k', fontsize=30)

    x_axs += 1
    if x_axs > 1:
        x_axs = 0
        y_axs += 1

plt.tight_layout()
plt.show()

save_figure(plot = fig, destination_path=figure_destination_path, figure_name= figure_name)

In [ ]:
#--------------variables for code-------------
figure_name="Immunogenicity_HD_Gold_allIncludingSpikePositive"
data_HD= data[(data["case/control"]=="HD case")]
data_HD_previousCOVID = data_HD[data_HD["History of COVID-19 infection"] == "YES"]
data_HD_NopreviousCOVID

# Define column sets and other parameters
data_4figure = data_HD_gold

column_sets = [
    ['IgGPre', 'IgGpost1or2D1', 'IgGpost21D1', 'IgGpost1or2D2', 'IgGpost21D2', 'IgGpost60D2'],
    ['NeutrolizingPre', 'Neutrolizing1or2D1', 'Neutrolizing21D1', 'Neutrolizing1or2D2', 'Neutrolizing21D2', 'Neutrolizing60D2'],
    ['AntiSpikepre', 'AntiSpike1or2D1', 'AntiSpike21D1', 'AntiSpike1or2D2', 'AntiSpike21D2', 'AntiSpike60D2'],
    ['AntiRBDpre', 'AntiRBD1or2D1', 'AntiRBD21D1', 'AntiRBD1or2D2', 'AntiRBD21D2', 'AntiRBD60D2']
]
name_set = ['IgG', 'Neutralizing Ab', 'Anti Spike Ab', 'Anti RBD']
unit_set = ['Index', 'micg/ml', 'RU/mL', 'Index']
cutoff_set = [1.1, 2.5, 8, 1.1]
xticks = ['Pre', '1or2D1', '21D1', '1or2D2', '21D2', '60D2']
color_palete=["#00D5E5", "#00D5E5", "#3C93C2", "#00D5E5", "#3C93C2", "#3C93C2"]

#--------------code-------------
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel, wilcoxon, sem
import itertools




# Define a function to calculate 95% confidence interval
def calc_95ci(data1, data2):
    diff = data1 - data2
    mean_diff = np.mean(diff)
    ci = 1.96 * sem(diff)
    return mean_diff - ci, mean_diff + ci


fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(40, 40), sharey=False)
x_axs = 0
y_axs = 0

for i, columns in enumerate(column_sets):
    melted_data = pd.melt(data_4figure, value_vars=columns, var_name='Timepoint', value_name='Value')

    sns.boxplot(x='Timepoint', y='Value', data=melted_data,
                ax=axs[x_axs, y_axs],
                palette=["#00D5E5", "#00D5E5", "#00D5E5", "#00D5E5", "#00D5E5", "#00D5E5"],
                linewidth=5)


    axs[x_axs, y_axs].axhline(y=cutoff_set[i], linestyle=':', color='grey', linewidth=7)

    sns.stripplot(x='Timepoint', y='Value', data=melted_data,
                  ax=axs[x_axs, y_axs],
                  palette=["black"],
                  size=13,
                  alpha=0.5)
    axs[x_axs, y_axs].set_title(f'{name_set[i]}', fontsize=55)
    axs[x_axs, y_axs].set_xlabel(' ', fontsize=35)
    axs[x_axs, y_axs].set_ylabel(f'{unit_set[i]}', fontsize=35)
    axs[x_axs, y_axs].tick_params(axis='both', labelsize=30)
    axs[x_axs, y_axs].set_xticklabels(xticks)

    bottom, top = axs[x_axs, y_axs].get_ylim()
    y_range = top - bottom

    combinations = itertools.combinations(columns, 2)
    significant_combinations = []
    for combination in combinations:
        # Extract data for the two time points
        valid_comb_data = data.dropna(subset=[combination[0], combination[1]])
        tp1 = valid_comb_data[combination[0]]
        tp2 = valid_comb_data[combination[1]]
        t_stat, p_value = wilcoxon(tp1, tp2)
        ci_low, ci_high = calc_95ci(tp1, tp2)
        significant_combinations.append([combination, p_value, (ci_low, ci_high)])

    for j, significant_combination in enumerate(significant_combinations):
        x1 = columns.index(significant_combination[0][0])
        x2 = columns.index(significant_combination[0][1])
        level = len(significant_combinations) - j
        bar_height = (y_range * 0.1 * level) + top
        bar_tips = bar_height - (y_range * 0.02)
        axs[x_axs, y_axs].plot(
            [x1, x1, x2, x2],
            [bar_tips, bar_height, bar_height, bar_tips], lw=1, c='k'
        )

        p = significant_combination[1]
        if p < 0.001:
            sig_symbol = '***'
        elif p < 0.01:
            sig_symbol = '**'
        elif p < 0.05:
            sig_symbol = '*'
        else:
            sig_symbol = 'NS'

        CI95 = significant_combination[2]
        text4plot = f"[{CI95[0]:.2f}, {CI95[1]:.2f}] {sig_symbol}"
        text_height = bar_height + (y_range * 0.01)
        axs[x_axs, y_axs].text((x1 + x2) / 2, text_height, text4plot, ha='center', va='bottom', c='k', fontsize=30)

    x_axs += 1
    if x_axs > 1:
        x_axs = 0
        y_axs += 1

plt.tight_layout()
plt.show()

save_figure(plot = fig, destination_path=figure_destination_path, figure_name= figure_name)